# Setup

In [1]:
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from circuits.sparse.compile import compiled_from_io
from circuits.neurons.core import const
from circuits.examples.simple_example import and_gate
from circuits.dense.mlp import StepMLP

# Create the standard language model

In [2]:
model_name = "gpt2"

standard_model = AutoModelForCausalLM.from_pretrained(model_name)

config = AutoConfig.from_pretrained(model_name)
model_dim = config.n_embd
vocab_size = config.vocab_size

print(f"Loaded '{model_name}'. Model dimension: {model_dim}, Vocab size: {vocab_size}")

Loaded 'gpt2'. Model dimension: 768, Vocab size: 50257


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
test_input = tokenizer(["The best animal in the world is"], return_tensors='pt')
generated_ids = standard_model.generate(**test_input, max_length=30)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'The best animal in the world is a dog.\n\nThe best animal in the world is a dog.\n\nThe best animal in the world'

# Create the circuit

In [5]:
sample_input = const("111")
sample_output = [and_gate(sample_input)]

graph = compiled_from_io(inputs=sample_input, outputs=sample_output)

In [8]:
from circuits.dense.mlp import Matrices


Matrices.from_graph(graph)

Matrices(mlist=[tensor([[ 1.,  0.,  0.,  0.],
        [-2.,  1.,  1.,  1.]])], dtype=torch.int32)

In [6]:
mlp_circuit = StepMLP.from_graph(graph)

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1